In [1]:
# load modules
import gc
from  psutil import virtual_memory
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
import pickle
from sklearn.metrics import make_scorer, mean_squared_error

import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

def print_memory():
    print('used:', virtual_memory().used / 1024. / 1024  / 1024)

stop = stopwords.words('english') + list(string.punctuation) + ['rm', '[rm]']
stop.remove('not')

print_memory()
print('Modules load OK!')

used: 12.752765655517578
Modules load OK!


In [2]:
def category_split(text):
    if text == 'missing':
        return ("missing", "missing", "missing")
    else:
        try:
            text_split = text.split("/")
            return text_split
        except:
            print(text)
            return ("missing", "missing", "missing")
        
negatives = {
    "didn't": "did_not",
    "couldn't": "could_not",
    "don't": "do_not",
    "wouldn't": "would_not",
    "doesn't": "does_not",
    "wasn't": "was_not",
    "weren't": "were_not",
    "shouldn't":"should_not",
    "isn't": "is_not",
    "aren't": "are_not",
}

regex = re.compile('[' +re.escape(string.punctuation) + '0-9\\r\\t\\n]')

def tokenize_text(text, treshold=2):
    return [i for i in text.split(' ') if i not in stop and len(i) > treshold]
    
def len_text(text, treshold=2):
    return len(tokenize_text(text, treshold))


def clear_text(text, treshold=2):
    text = regex.sub(" ", text.lower()) # remove punctuation
    text = re.sub("\s\s+" , " ", text) # remove multiple spacas
    for k, v in negatives.items():
        text = text.replace(k, v)
    return text

def rmsle(h, y): 
    """
    Compute the Root Mean Squared Log Error for hypothesis h and targets y

    Args:
        h - numpy array containing predictions with shape (n_samples, n_targets)
        y - numpy array containing targets with shape (n_samples, n_targets)
    """
    return np.sqrt(np.square(np.log(h + 1) - np.log(y + 1)).mean())


rmsle_scorer = make_scorer(rmsle, greater_is_better=False)

def rmse_my(h, y): 
    """
    Compute the Root Mean Squared Log Error for hypothesis h and targets y

    Args:
        h - numpy array containing predictions with shape (n_samples, n_targets)
        y - numpy array containing targets with shape (n_samples, n_targets)
    """
    return np.sqrt(np.square(h - y).mean())


rmse_my_scorer = make_scorer(rmse_my, greater_is_better=False)

In [3]:
# Load data

count_train = 1482535
sample_subm = pd.DataFrame({'test_id':np.arange(693359), 'price':0}, columns=['test_id', 'price'])

#defite target, log target and remove it from train data
y_target = pd.read_csv('data/train.tsv', sep='\t', index_col=0, usecols=['train_id', 'price']).price.values
y_target_log = np.log1p(y_target)

/usr/local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
all_data = pd.concat([pd.read_csv('data/train.tsv', sep='\t', index_col=0).drop(['price'], axis=1), pd.read_csv('data/test.tsv', sep='\t', index_col=0)])

print('size of all_data and data_sample dataset: ', all_data.shape)

print_memory()
print('Data load')

# # Preprosessing data

# ### Remove NA
# remove NA in item_description - 4 rows
all_data['item_description'] = all_data['item_description'].fillna(value='No description yet')

# fill NA in brand_name and category_name by 'missing'
all_data[['name', 'category_name', 'brand_name']] = all_data[['name', 'category_name', 'brand_name']].fillna(value='missing', axis=1)

print_memory()
print('NA removed.')

# ### Preprocess categorical features

# create 3 categorical variables from category_name



all_data['category'], all_data['sub_cat1'], all_data['sub_cat2'] = zip(*all_data['category_name'].apply(category_split))
all_data.drop(['category_name'], axis=1, inplace=True)

label_enc = LabelEncoder()
all_data['brand_name'] = label_enc.fit_transform(all_data[['brand_name']])
all_data['category'] = label_enc.fit_transform(all_data[['category']])
all_data['sub_cat1'] = label_enc.fit_transform(all_data[['sub_cat1']])
all_data['sub_cat2'] = label_enc.fit_transform(all_data[['sub_cat2']])

print_memory()
print('categorical variables OK!')


# ### Preprocess text features

# In[10]:




all_data['item_description'] = all_data.item_description.apply(clear_text)
all_data['name'] = all_data.name.apply(clear_text)

print_memory()
print('data clear!')


all_data['word_count_desc'] = all_data.item_description.apply(len_text)
all_data['word_count_name'] = all_data.name.apply(len_text)

gc.collect()
print_memory()
print('text feature ok!')




with open('X_numeric.pickle', 'wb') as f:
    pickle.dump(all_data[['item_condition_id', 'shipping', 'word_count_desc', 'word_count_name',
                      'brand_name', 'category', 'sub_cat1', 'sub_cat2']], f, pickle.HIGHEST_PROTOCOL)

with open('data_description.pickle', 'wb') as f:
    pickle.dump(all_data['item_description'], f, pickle.HIGHEST_PROTOCOL)   
    
with open('data_name.pickle', 'wb') as f:
    pickle.dump(all_data['name'], f, pickle.HIGHEST_PROTOCOL)   

del all_data
gc.collect()
print_memory()

In [4]:
tf_idf = TfidfVectorizer(min_df=0.0005, ngram_range=(1, 3), tokenizer=tokenize_text)

with open('data_description.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    X_description = tf_idf.fit_transform(pickle.load(f))
    print('description tf-idf ok!')

print_memory()

# tsvd_desc = TruncatedSVD(n_components=200)
# X_description_svd = tsvd_desc.fit_transform(X_description)
# print('TruncatedSVD description OK!')

# del X_description
# gc.collect()
# print_memory()

with open('data_name.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    X_name = tf_idf.fit_transform(pickle.load(f))
    print('all tf-idf ok!')

# tsvd_name = TruncatedSVD(n_components=30)
# X_name_svd = tsvd_name.fit_transform(X_name)

# print('TruncatedSVD all OK!')
# del X_name
# gc.collect()


with open('X_numeric.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    X_numeric = MinMaxScaler().fit_transform(pickle.load(f))

description tf-idf ok!
used: 10.044845581054688
all tf-idf ok!


In [5]:
from scipy.sparse import csr_matrix, hstack
train = hstack((X_description[:count_train,], X_name[:count_train,], X_numeric[:count_train,])).tocsr()
test = hstack((X_description[count_train:,], X_name[count_train:,], X_numeric[count_train:,])).tocsr()

print('size of finish train and test dataset: ', train.shape, test.shape)

size of finish train and test dataset:  (1482535, 7033) (693359, 7033)


In [6]:
train_X, valid_X, train_y, valid_y = train_test_split(train, y_target_log, random_state=42, test_size=0.15)
print(train_X.shape, valid_X.shape, len(train_y), len(valid_y))

(1260154, 7033) (222381, 7033) 1260154 222381


In [7]:
# del X_description_svd
# del X_name_svd
# del X_numeric
# gc.collect()
# print_memory()

# print('size of finish train and test dataset: ', train.shape, test.shape)

#train_X, valid_X, train_y, valid_y = train_test_split(data_ok, y_target_log, random_state=42, test_size=0.05)


param_grid_linear = {'normalize':[True]}
cv_linear = GridSearchCV(LinearRegression(), param_grid_linear, scoring=rmse_my_scorer, cv=5, n_jobs=-1, verbose=1)
cv_linear.fit(train_X, train_y)

print('linear CV score:', cv_linear.best_score_)

preds_linear = cv_linear.predict(valid_X)

print('score test:', rmse_my(preds_linear, valid_y))

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  2.1min remaining:  3.1min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.4min finished


linear CV score: -0.545692403092
score test: 0.547312447432


In [43]:
print('score test:', rmse_my(preds_linear, valid_y))

score test: 0.547312446604


In [4]:
#preds_linear = cv_linear.best_estimator_.predict(test)
sample_subm.price = np.expm1(cv_linear.best_estimator_.predict(test))
sample_subm.to_csv("submission_linear_CV.csv", index=False)

description tf-idf ok!
used: 8.472480773925781
TruncatedSVD description OK!
used: 11.380382537841797
all tf-idf ok!
TruncatedSVD all OK!
size of finish train and test dataset:  (1482535, 238) (693359, 238)
used: 6.52081298828125
size of finish train and test dataset:  (1482535, 238) (693359, 238)
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.6min finished


linear CV score: -0.415891098339


linear CV score: -0.422297011147 for 150

In [26]:
param_grid_linear = {'normalize':[True]}
cv_linear = GridSearchCV(LinearRegression(), param_grid_linear, scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=1)
cv_linear.fit(train_X, train_y)

print('linear CV score:', cv_linear.best_score_)

#preds_linear = cv_linear.best_estimator_.predict(test)
print('score test:', mean_squared_error(cv_linear.best_estimator_.predict(valid_X), valid_y))

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.4min finished


linear CV score: -0.415576887713
score test: 0.417793518429


In [45]:
import lightgbm as lgb

d_train = lgb.Dataset(train_X, label=train_y)
d_valid = lgb.Dataset(valid_X, label=valid_y)
watchlist = [d_train, d_valid]

params = {
    'learning_rate': 0.5,
    'application': 'regression',
    'max_depth': 3,
    'num_leaves': 60,
    'verbosity': -1,
    'metric': 'RMSE',
    'data_random_seed': 1,
    'bagging_fraction': 0.5,
    'nthread': 4
}

params2 = {
    'learning_rate': 1,
    'application': 'regression',
    'max_depth': 3,
    'num_leaves': 140,
    'verbosity': -1,
    'metric': 'RMSE',
    'data_random_seed': 2,
    'bagging_fraction': 1,
    'nthread': 4
}

model = lgb.train(params, train_set=d_train, num_boost_round=8000, valid_sets=watchlist, \
                    early_stopping_rounds=50, verbose_eval=500) 
predsL = model.predict(valid_X)

print('score test:', rmse_my(predsL, valid_y))

Training until validation scores don't improve for 50 rounds.
[500]	training's rmse: 0.517998	valid_1's rmse: 0.524789
[1000]	training's rmse: 0.497909	valid_1's rmse: 0.509423
[1500]	training's rmse: 0.486349	valid_1's rmse: 0.501498
[2000]	training's rmse: 0.47864	valid_1's rmse: 0.497125
[2500]	training's rmse: 0.47132	valid_1's rmse: 0.492829
[3000]	training's rmse: 0.465945	valid_1's rmse: 0.490404
[3500]	training's rmse: 0.460989	valid_1's rmse: 0.488338
[4000]	training's rmse: 0.456488	valid_1's rmse: 0.486767
[4500]	training's rmse: 0.452069	valid_1's rmse: 0.485054
Early stopping, best iteration is:
[4563]	training's rmse: 0.451499	valid_1's rmse: 0.484902
score test: 0.484901830891


In [47]:
alpha = 0.5
for alpha in np.linspace(0, 1, 50):
    print('alpha=', alpha, ' score test:', rmse_my(predsL * alpha + (1 - alpha) * preds_linear, valid_y))

alpha= 0.0  score test: 0.547312446604
alpha= 0.0204081632653  score test: 0.544871788463
alpha= 0.0408163265306  score test: 0.54247203276
alpha= 0.0612244897959  score test: 0.540113724692
alpha= 0.0816326530612  score test: 0.537797409519
alpha= 0.102040816327  score test: 0.535523632138
alpha= 0.122448979592  score test: 0.533292936651
alpha= 0.142857142857  score test: 0.531105865902
alpha= 0.163265306122  score test: 0.528962961008
alpha= 0.183673469388  score test: 0.526864760875
alpha= 0.204081632653  score test: 0.524811801691
alpha= 0.224489795918  score test: 0.522804616417
alpha= 0.244897959184  score test: 0.520843734251
alpha= 0.265306122449  score test: 0.518929680092
alpha= 0.285714285714  score test: 0.517062973982
alpha= 0.30612244898  score test: 0.515244130539
alpha= 0.326530612245  score test: 0.513473658386
alpha= 0.34693877551  score test: 0.511752059563
alpha= 0.367346938776  score test: 0.510079828937
alpha= 0.387755102041  score test: 0.508457453598
alpha= 0.4

In [28]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=.05, copy_X=True, fit_intercept=True, max_iter=100,
      normalize=False, random_state=101, solver='auto', tol=0.001)
model.fit(train_X, train_y)

preds_ridge = model.predict(valid_X)

print('score test:', rmse_my(preds_ridge, valid_y))

score test: 0.547315646056


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [51]:
#train_X, valid_X, train_y, valid_y = train_test_split(train, y_target_log, random_state=101, test_size=0.15)
print(train_X.shape, valid_X.shape, len(train_y), len(valid_y))

d_train = lgb.Dataset(train_X, label=train_y)
d_valid = lgb.Dataset(valid_X, label=valid_y)
watchlist = [d_train, d_valid]
model = lgb.train(params2, train_set=d_train, num_boost_round=8000, valid_sets=watchlist, \
                    early_stopping_rounds=50, verbose_eval=500) 
predsL2 = model.predict(valid_X)

print('score test:', rmse_my(predsL2, valid_y))

(1260154, 7033) (222381, 7033) 1260154 222381
Training until validation scores don't improve for 50 rounds.
[500]	training's rmse: 0.514086	valid_1's rmse: 0.525975
[1000]	training's rmse: 0.492521	valid_1's rmse: 0.511341
[1500]	training's rmse: 0.480879	valid_1's rmse: 0.506133
Early stopping, best iteration is:
[1780]	training's rmse: 0.474929	valid_1's rmse: 0.503866
score test: 0.503866340833


In [52]:
for alpha in np.linspace(0, 1, 50):
    print('alpha=', alpha, ' score test:', rmse_my(predsL * alpha + (1 - alpha) * predsL2, valid_y))

alpha= 0.0  score test: 0.503866340833
alpha= 0.0204081632653  score test: 0.502681790929
alpha= 0.0408163265306  score test: 0.501528075705
alpha= 0.0612244897959  score test: 0.500405408433
alpha= 0.0816326530612  score test: 0.499313998541
alpha= 0.102040816327  score test: 0.498254051434
alpha= 0.122448979592  score test: 0.497225768321
alpha= 0.142857142857  score test: 0.496229346044
alpha= 0.163265306122  score test: 0.495264976905
alpha= 0.183673469388  score test: 0.494332848497
alpha= 0.204081632653  score test: 0.493433143535
alpha= 0.224489795918  score test: 0.492566039689
alpha= 0.244897959184  score test: 0.491731709423
alpha= 0.265306122449  score test: 0.490930319832
alpha= 0.285714285714  score test: 0.490162032484
alpha= 0.30612244898  score test: 0.489427003269
alpha= 0.326530612245  score test: 0.488725382244
alpha= 0.34693877551  score test: 0.48805731349
alpha= 0.367346938776  score test: 0.487422934969
alpha= 0.387755102041  score test: 0.486822378385
alpha= 0.4

In [11]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer

#import xgboost as xgb
import lightgbm as lgb

In [ ]:
def objective(params):
    params = {
        'num_leaves': int(params['num_leaves']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'n_estimators': int(params['n_estimators']),
        'learning_rate': '{:.3f}'.format(params['learning_rate']),
    }
    
    clf = lgb.LGBMRegressor(
        #n_estimators=500,
        #learning_rate=0.01,
        **params
    )
    
    score = cross_val_score(clf, train, y_target_log, scoring=rmse_my_scorer, cv=3).mean()
    print("Score {:.3f} params {}".format(score, params))
    return score

space = {
    'num_leaves': hp.quniform('num_leaves', 8, 128, 2),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'n_estimators': hp.quniform('n_estimators', 50, 500, 25),
    'learning_rate': hp.uniform('learning_rate', 0.1, 1.0)
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
           verbose=1)

print("Hyperopt estimated optimum {}".format(best))

Score -0.509 params {'colsample_bytree': '0.549', 'learning_rate': '0.624', 'num_leaves': 62, 'n_estimators': 75}
Score -0.542 params {'colsample_bytree': '0.904', 'learning_rate': '0.608', 'num_leaves': 20, 'n_estimators': 50}
Score -0.483 params {'colsample_bytree': '0.852', 'learning_rate': '0.349', 'num_leaves': 74, 'n_estimators': 450}
Score -0.515 params {'colsample_bytree': '0.370', 'learning_rate': '0.820', 'num_leaves': 102, 'n_estimators': 475}


In [15]:
cls_rf = RandomForestRegressor(n_estimators=50, max_depth=10, n_jobs=-1, criterion='mse', verbose=1)


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 16.5min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 21.7min finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.3s finished


score test: 0.632517326612


In [27]:
from sklearn.linear_model import PassiveAggressiveRegressor

cls_pass = PassiveAggressiveRegressor(verbose=1)
cls_pass.fit(train_X, train_y)

preds_pass = cls_pass.predict(valid_X)
print('score test:', rmse_my(preds_pass, valid_y))

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.passive_aggressive.PassiveAggressiveRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


-- Epoch 1
Norm: 29.48, NNZs: 7033, Bias: 2.987773, T: 1260154, Avg. loss: 0.489053
Total training time: 0.62 seconds.
-- Epoch 2
Norm: 29.97, NNZs: 7033, Bias: 3.023569, T: 2520308, Avg. loss: 0.487582
Total training time: 1.23 seconds.
-- Epoch 3
Norm: 30.35, NNZs: 7033, Bias: 2.952076, T: 3780462, Avg. loss: 0.486805
Total training time: 1.84 seconds.
-- Epoch 4
Norm: 30.11, NNZs: 7033, Bias: 2.970759, T: 5040616, Avg. loss: 0.487350
Total training time: 2.48 seconds.
-- Epoch 5
Norm: 30.26, NNZs: 7033, Bias: 3.003744, T: 6300770, Avg. loss: 0.487546
Total training time: 3.09 seconds.
score test: 0.685491348431
